In [6]:
scope.dis()
target.dis()

In [7]:
import chipwhisperer as cw
import sys
sys.path.append("../imports")
from helpers import *
from scheme import Scheme
from taskers import *

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()

In [43]:
load(scope, "masked-decode-inside", env)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 12355 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 12355 bytes


In [99]:
scope.default_setup()
scope.adc.samples=24400
scope.clock.adc_src = "clkgen_x1"
scope.adc.decimate = 2

In [98]:
#load(scope, "masked-decode-inside", env)
target.flush()
scope.arm()
w = [1, 1, 1, 0 ,1, 0] + [0 for i in range(250)]
target.simpleserial_write('p', get_bytes(w))
ret = scope.capture()
if ret:
    print("ERROR")
traces_ = scope.get_last_trace()
db = target.simpleserial_read('r', 16)
print(scope.adc.trig_count)


63139


In [13]:

# function
def setup():
    # runs once in the start -> prints returned
    # load(scope, "../c-files/???", env)
    load(scope, "masked-decode-inside", env)
    # print(date_str())
    pass

print("bring back randomness")
current_key = None
current_rand = None
def execute(index, op, t):
    global current_key, current_rand

    FI_HD = [0 for i in range(127)]
    FI_PUF = [0 for i in range(127)]
    rand = [random.randrange(1) for _ in range(64)]
    if op == "00":
        p_temp = p0.sample(byte=False)
        k_temp = k0
    elif op == "01":
        p_temp = p0.sample(byte=False)
        k_temp = k1
    elif op == "00_deg":
        rand = [random.randrange(1) for _ in range(64)]
        p_temp = p0.sample(byte=False)
        k_temp = k0
    elif op == "11_deg":
        rand = [random.randrange(1) for _ in range(64)]
        p_temp = p1.sample(byte=False)
        k_temp = k1
    elif op == "11":
        p_temp = p1.sample(byte=False)
        k_temp = k1
    elif op == "22":
        p_temp = p2.sample(byte=False)
        k_temp = k2
    elif op == "22_deg":
        rand = [random.randrange(1) for _ in range(64)]
        p_temp = p2.sample(byte=False)
        k_temp = k2
    elif op == "33":
        p_temp = p3.sample(byte=False)
        k_temp = k3
    elif op == "04":
        p_temp = p0.sample(byte=False)
        k_temp = k4
    elif op == "00_FI_HD_64":
        p_temp = p0.sample(byte=False)
        k_temp = k0
        FI_HD[63] ^= 1
    elif op == "00_FI_HD_64_deg":
        rand = [random.randrange(1) for _ in range(64)]
        p_temp = p0.sample(byte=False)
        k_temp = k0
        FI_HD[63] ^= 1
    else:
        raise "ERROR"
        
    
    mem = xor(p_temp, s.encode(k_temp), FI_HD)
    posture(target,'k', get_bytes(mem), 16)
    p_temp = xor(FI_PUF, p_temp)
    current_key = k_temp
    current_rand = rand
    return get_bytes(p_temp + [0] + s.encode(rand))

def ending(fdb, op, t):
    
    if current_key != xor(current_rand,get_bits(fdb)[:64]):
        print(s.encode(current_key))
        print(current_key)
        print(get_bits(fdb))
        print("ahHAHAHA")
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    pass


s = Scheme() # scheme execution helper


p0 = [random.randrange(2) for _ in range(127)]
k0 = [random.randrange(2) for _ in range(64)]


p1 = [random.randrange(2) for _ in range(127)]
k1 = [random.randrange(2) for _ in range(64)]


k2 = [random.randrange(2) for _ in range(64)]
p2 = xor(p0, s.encode(k2), s.encode(k0))


HWHD0 = sum(xor(p0, s.encode(k0)))
p3 = [random.randrange(2) for _ in range(127)]
k3 = [random.randrange(2) for _ in range(64)]
HWHD3 = sum(xor(p3, s.encode(k3)))
# try to get the 4 32 bit sections to have the same weight?
while HWHD3 != HWHD0 or sum(p3) != sum(p0):
    p3 = [random.randrange(2) for _ in range(127)]
    k3 = [random.randrange(2) for _ in range(64)]
    HWHD3 = sum(xor(p3, s.encode(k3)))


k4 = [random.randrange(2) for _ in range(64)]
HWHD4 = sum(xor(p0, s.encode(k4)))
while HWHD4 != HWHD0:
    k4 = [random.randrange(2) for _ in range(64)]
    HWHD4 = sum(xor(p0, s.encode(k4)))


assert xor(p2, s.encode(k2)) == xor(p0, s.encode(k0))
assert sum(xor(p0, s.encode(k0))) == sum(xor(p3, s.encode(k3)))
assert sum(p3) == sum(p0)
assert sum(xor(p0, s.encode(k0))) == sum(xor(p0, s.encode(k4)))

unstable = None
p0 = PUF(p0, unstable)
p1 = PUF(p1, unstable)
p2 = PUF(p2, unstable)
p3 = PUF(p3, unstable)

env = ENV()
# operation settings
prefix_name = "masked"
env.root = "D:/programs/traces/MASKED"
env.execute = execute
env.ending = ending
env.jupyter = "masked_new" # current notebook name
# 33 removed
env.operations = operations = ["00_deg", "11_deg", "22_deg", "00_FI_HD_64_deg"] # operations list
inv = lambda x: operations.index(x)
env.subfolder = "full_nj" # subfolder name
env.TRACES_NUM = 1000 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.segments = 3 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = ""
env.title = ""
env.name = prefix_name
env.readme = ""
env.graph = {"type":"time", "xlines": [950, 60900]}


scope.clock.adc_src = "clkgen_x1"
scope.adc.decimate = 1

# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()

file_names = [f"file{i}" for i in range(2)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)


i = 0

i += 1
env.name = prefix_name+f"_{i}"
env.title = "Codeword masking S_0=\{PUF_0, k_0\} vs S_1=\{PUF_1, k_1\}"
env.desc = "mask <= 0"
create_tasker(env, file_names, [inv("00_deg"), inv("11_deg")], f"group{i}")

i += 1
env.name = prefix_name+f"_{i}"
env.title = "Codeword masking S_0=\{PUF_0, k_0\} vs S_1=\{PUF_1, k_1\}"
env.desc = "HD_1 = HD_2, mask <= 0"
create_tasker(env, file_names, [inv("00_deg"), inv("22_deg")], f"group{i}")

i += 1
env.name = prefix_name+f"_{i}"
env.title = "Codeword masking S_0=\{PUF_0, k_0\} vs S_1=\{PUF_1, k_1\} + "
env.desc = "bitflip FI on HD_1[64], mask <= 0"
create_tasker(env, file_names, [inv("00_deg"), inv("00_FI_HD_64_deg")], f"group{i}")




env.name = prefix_name
#remove_files(env, file_names)
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())


#print(s.encode(current_key))
#print(current_key)
#print(get_bits(fdb))

bring back randomness


KeyboardInterrupt: 

In [ ]:
execute mission
# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()
for i in range(1,6):
    status = traces("file_name", 0)
split_file(["file_name"], [0, 1], "result")
remove_files(["file_name"])
generate_file("start_time", "operations")
print("done")